In [1]:
using Plots
using .Threads

println("Number of threads $(nthreads())")

Number of threads 8


In [2]:
a = zeros(Int, 8)

@threads for i = eachindex(a)
    a[i] = threadid()
end

a

8-element Vector{Int64}:
 1
 5
 3
 6
 4
 8
 7
 2

# Gray-Scott system

$$
\begin{aligned}
\frac{\partial u}{\partial t} & = D\_u \Delta u - uv^2 + F(1-u) \\\\ 
\frac{\partial v}{\partial t} & = D\_v \Delta v + uv^2 - (F+k)v 
\end{aligned} 
$$

## Laplacian 

$$
\Delta u\_{i,j} \approx u\_{i,j-1} + u\_{i-1,j} -4u\_{i,j} + u\_{i+1, j} + u\_{i, j+1} 
$$

- Euler scheme is used to integrate the time derivative.  

## Initial conditions

- ``u`` is 1 everywhere and ``v`` is 0 in the domain except in a square zone where ``v = 0.25`` and ``u = 0.5``. This square located in the center of the domain is  $[0, 1]\times[0,1]$ with a size of ``0.2``.

In [3]:
const Dᵤ = .1
const Dᵥ = .05
const F = 0.0545
const k = 0.062

function init(n)
 
    u = ones((n+2,n+2))
    v = zeros((n+2,n+2))
    
    x, y = LinRange(0, 1, n+2), LinRange(0, 1, n+2)

    for j in eachindex(y), i in eachindex(x)
        if (0.4<x[i]) && (x[i]<0.6) && (0.4<y[j]) && (y[j]<0.6)
            u[i,j] = 0.50
            v[i,j] = 0.25
        end
    end
        
    return u, v
    
end

init (generic function with 1 method)

## Serial function

In [4]:
function grayscott!( u, v, Δu, Δv)

    for c = axes(Δu, 2)
        c1 = c + 1
        c2 = c + 2
        for r = axes(Δu, 1)
            r1 = r + 1
            r2 = r + 2
            Δu[r,c] = u[r1,c2] + u[r1,c] + u[r2,c1] + u[r,c1] - 4*u[r1,c1]
            Δv[r,c] = v[r1,c2] + v[r1,c] + v[r2,c1] + v[r,c1] - 4*v[r1,c1]
        end
    end

    for c = axes(Δu, 2)
        c1 = c + 1
        for r = axes(Δu, 1)
            r1 = r + 1  
            uvv = u[r1,c1]*v[r1,c1]*v[r1,c1]
            u[r1,c1] +=  Dᵤ * Δu[r,c] - uvv + F*(1 - u[r1,c1])
            v[r1,c1] +=  Dᵥ * Δv[r,c] + uvv - (F + k)*v[r1,c1]
        end
    end

end

grayscott! (generic function with 1 method)

In [5]:
function create_animation( grayscott_function, n = 300, maxiter = 10_000)
    u, v = init(n)
    Δu = zeros(n, n)
    Δv = zeros(n, n)
    options = (aspect_ratio = :equal, axis = nothing, 
               legend = :none, framestyle = :none)
    anim = @animate for t in 1:maxiter
        grayscott_function(u, v, Δu, Δv)
        heatmap(u; options...)
    end every 100
    return anim
end
anim = create_animation( grayscott!, 300)
gif(anim, "anim.gif", fps = 15)

GKS: libtiff.so.6: cannot open shared object file: No such file or directory
Output file #0 does not contain any stream


LoadError: failed process: Process(`[4m/home/pnavaro/.julia/artifacts/5b83972689fb7dea5e89326f1c0ba60d68e962fb/bin/ffmpeg[24m [4m-v[24m [4m16[24m [4m-i[24m [4m/tmp/jl_U4Mn6O/%06d.png[24m [4m-vf[24m [4mpalettegen=stats_mode=full[24m [4m-y[24m [4m/tmp/jl_U4Mn6O/palette.bmp[24m`, ProcessExited(1)) [1]


# Simulation

In [19]:
function run_simulation( grayscott_function, n = 300, maxiter = 10_000)
    u, v = init(n)
    Δu = zeros(n, n)
    Δv = zeros(n, n)
    options = (aspect_ratio = :equal, axis = nothing, 
               legend = :none, framestyle = :none)
    for t in 1:maxiter
        grayscott_function(u, v, Δu, Δv)
        #heatmap(u; options...)
    end #every 100
    #return anim
end
run_simulation( grayscott!, 600, 1)
@time anim = run_simulation( grayscott!, 600)
#gif(anim, "anim1.gif", fps = 15)

  9.438768 seconds (8 allocations: 11.023 MiB)


In [20]:
using .Threads

function grayscott_threads!( u, v, Δu, Δv)

    n = size(Δu,1)

    @threads for c = axes(Δu, 2)
        c1 = c + 1
        c2 = c + 2
        for r = axes(Δu, 1)
            r1 = r + 1
            r2 = r + 2
            Δu[r,c] = u[r1,c2] + u[r1,c] + u[r2,c1] + u[r,c1] - 4*u[r1,c1]
            Δv[r,c] = v[r1,c2] + v[r1,c] + v[r2,c1] + v[r,c1] - 4*v[r1,c1]
        end
    end

    @threads for c = axes(Δu, 2)
        c1 = c + 1
        for r = axes(Δu, 1)
            r1 = r + 1  
            uvv = u[r1,c1]*v[r1,c1]*v[r1,c1]
            u[r1,c1] +=  Dᵤ * Δu[r,c] - uvv + F*(1 - u[r1,c1])
            v[r1,c1] +=  Dᵥ * Δv[r,c] + uvv - (F + k)*v[r1,c1]
        end
    end

end
run_simulation( grayscott_threads!, 600, 1)
@time anim = run_simulation( grayscott_threads!, 600)
#gif(anim, "anim2.gif", fps = 15)

  6.001148 seconds (820.13 k allocations: 96.171 MiB, 0.43% gc time)


## Vectorized version

In [21]:
function grayscott_vectorized!(u, v, Δu, Δv, uvv)

   @views @. begin 
       Δu =(u[begin:end-2,2:end-1] + u[begin+1:end-1,1:end-2] + u[begin+1:end-1, begin+2:end] 
          + u[begin+2:end,2:end-1] - 4 * u[begin+1:end-1, begin+1:end-1] )
    
       Δv = (v[begin:end-2,2:end-1] + v[begin+1:end-1,1:end-2] + v[begin+1:end-1, begin+2:end] 
          + v[begin+2:end,begin+1:end-1] - 4 * v[begin+1:end-1, begin+1:end-1] )
    
       uvv = u[begin+1:end-1,begin+1:end-1] * v[begin+1:end-1,begin+1:end-1] * v[begin+1:end-1,begin+1:end-1]
    
       u[begin+1:end-1,begin+1:end-1] +=  Dᵤ * Δu - uvv + F * (1 - u[begin+1:end-1,begin+1:end-1])
    
       v[begin+1:end-1,begin+1:end-1] +=  Dᵥ * Δv + uvv - (F + k) * v[begin+1:end-1,begin+1:end-1]
   end

end

function run_simulation_vectorized( n = 300, maxiter = 10_000)
    u, v = init(n)
    Δu = zeros(n, n)
    Δv = zeros(n, n)
    uvv = zeros(n, n)
    options = (aspect_ratio = :equal, axis = nothing, 
               legend = :none, framestyle = :none)
    # anim = @animate 
    for t in 1:maxiter
        grayscott_vectorized!(u, v, Δu, Δv, uvv)
        #heatmap(u; options...)
    end #every 100
    #return anim
end

run_simulation_vectorized( 500, 1)
@time anim = run_simulation_vectorized( 600)
#gif(anim, "anim3.gif", fps = 15)

 24.706782 seconds (10 allocations: 13.770 MiB)


In [ ]:
using CUDA

function run_on_gpu( n = 500, maxiter = 10_000)

    u0, v0 = init(n)

    u = CuArray(u0) # allocates on GPU
    v = CuArray(v0)

    Δu = CUDA.zeros(n, n)
    Δv = CUDA.zeros(n, n)
    uvv = CUDA.zeros(n, n)
    options = (aspect_ratio = :equal, axis = nothing, 
               legend = :none, framestyle = :none)
    
    for t in 1:maxiter
    
        grayscott_vectorized!(u, v, Δu, Δv, uvv)

    end

    return Array(u), Array(v)

end

CUDA.@time u, v  = run_on_gpu( 500)